In [1]:
using Pkg
Pkg.activate(".") # Activate saved environment with compatible depenency versions
Pkg.instantiate()
using Nemo
using SharedArrays
using CUDA
using LinearAlgebra
using Combinatorics
using Test
using DataFrames
using CSV


  Activating project at `d:\SeqFISH+Processing_project\UntanglingBarcodes\codebook_generation\get_RS_codebooks\local`


This notebook shows how to generate codebooks for seqFISH experiments using [Reed-Solomon Codes](https://en.wikipedia.org/wiki/Reed%E2%80%93Solomon_error_correction). Reed-Solomon are part of a special class of error-correcting codes called [Maximum Distance Separable codes](https://en.wikipedia.org/wiki/Singleton_bound#MDS_codes) (MDS code) which achieve equality in the [Singleton bound](https://en.wikipedia.org/wiki/Singleton_bound). This means that MDS codes acheive the maximum possible extra difference between their codewords from every redundant parity check symbol, and gain the the most possible robustless to error for the increased cost of encoding information with more symbols.

The number of codewords of a given weight in an MDS code weight is given by the the expression 

$(q-1)\binom{w}{n}\sum_{i=0}^{w-d}(-1)^i \binom{i}{w-1}q^{w-d-i}$

(Macwilliams and Sloan)

## First, enter the parameters for the Reed-Solomon Code that you want.

In [2]:
q = 13
deg = 1
nmk = 3
wmax = 4
wmin=2

2

In [3]:
function get_num_codewords_of_weight_vn(q,n,w,d)
    i = collect(0:(w-d))
    (q-1)*binomial(n,w)*sum(((-1) .^ i) .* (binomial.(w-1, i)) .* (q .^ ((w - d) .- i)))
end

get_num_codewords_of_weight_vn (generic function with 1 method)

Reed-Solomon codes are constructed using abstract algebra. In this construction, codewords of a Reed-Solomon code are polynomials from a [polynomial ring](https://en.wikipedia.org/wiki/Polynomial_ring) with coefficients from a [finite field](https://en.wikipedia.org/wiki/Finite_field). The code polynomials of a Reed-Solomon code are multiples of the generator polynomial of the Reed-Solomon code. We denote the order of the finite field as $q$ and the length of codewords in the field as $n$. In this notebook, we use the open source package, [Jump.jl](http://nemocas.github.io/Nemo.jl/latest/), to do abstract algebra computations. First, we define the algebraic objects that we are working with:

In [4]:
function def_RS_code(_q :: Int64, deg :: Int64, nmk :: Int64)
    @assert is_prime(_q)
    global q = _q
    global q_uint8 = UInt8(q)
    global n = q-1
    global k = (q-1) - nmk
    F, α = FiniteField(q, deg, "α")
    R, x = PolynomialRing(F, "x")
    RR =  ResidueRing(R, x^(q^deg-1)-1)
    gp = 1
    for i = 1:nmk
        gp = gp*RR(x - α^i)
    end
    return F, RR, R, gp, x, α
end


function cvt_fq_nmod_2_int(x::fq_nmod)
    if iszero(x)
        return 0
    end
    for i = 1:(q-1)
        if iszero(i+x)
            return q-i
        end
    end 
end

cvt_fq_nmod_2_int (generic function with 1 method)

In [5]:
F, RR, R, gp, x, α = def_RS_code(q,deg,nmk)

(Finite field of degree 1 over GF(13), Residue ring of univariate polynomial ring modulo x^12 + 12, Univariate polynomial ring in x over GF(13), x^3 + 12*x^2 + 4*x + 1, x, 2)

In [6]:
gp

x^3 + 12*x^2 + 4*x + 1

In [8]:
k

9

Get the generator and parity check matrices of the base Reed-Solomon code and print the parity check matrix.

In [9]:
gm_fq_band_form = fill(α-α, k, q-1)
for i in 1:k
    shift = gp*x^(i-1)
    gm_fq_band_form[i,:] = coeff.(shift.data, 0:(q-2))
end
gm_fqn = matrix_space(F, k,q-1)(gm_fq_band_form)
H_fq = [(α^(i[1]))^(i[2]-1) for i in CartesianIndices((q-k-1, q-1))]
Matrix(H_fq)

3×12 Matrix{fqPolyRepFieldElem}:
 1  2  4   8   3  6   12  11  9  5   10  7
 1  4  3   12  9  10  1   4   3  12  9   10
 1  8  12  5   1  8   12  5   1  8   12  5

Check that the generator and parity check matrices are indeed orthogonal to each other

In [10]:
@test all(iszero.(Matrix(gm_fqn)*transpose(H_fq)))

Test Passed

Get Parity check Matrices of singly and doubly extended code and print them.

In [11]:
ext1col = fill(F(0), q-k-1)
ext1col[end] = F(1)
ext2col = fill(F(0), q-k-1)
ext2col[1] = F(1)
H_fq_ext1 = hcat(ext1col, H_fq)

3×13 Matrix{fqPolyRepFieldElem}:
 0  1  2  4   8   3  6   12  11  9  5   10  7
 0  1  4  3   12  9  10  1   4   3  12  9   10
 1  1  8  12  5   1  8   12  5   1  8   12  5

In [12]:
H_fq_ext2 = hcat(ext2col, H_fq_ext1)

3×14 Matrix{fqPolyRepFieldElem}:
 1  0  1  2  4   8   3  6   12  11  9  5   10  7
 0  0  1  4  3   12  9  10  1   4   3  12  9   10
 0  1  1  8  12  5   1  8   12  5   1  8   12  5

Find the row-reduced echelon form (linear algebra terminology) or systematic form (coding theory terminology) of the parity check matrices, then print them.

In [13]:
H_fq_ext1_rref = rref(matrix_space(F, q-k-1, q)(H_fq_ext1))[2]
H_fq_ext2_rref = rref(matrix_space(F, q-k-1, q+1)(H_fq_ext2))[2]
Matrix(H_fq_ext1_rref)

3×13 Matrix{fqPolyRepFieldElem}:
 1  0  0  11  11  6   3  7   2  10  8   5   2
 0  1  0  5   4   10  2  10  5  2   11  11  4
 0  0  1  6   2   3   2  1   3  10  10  6   8

In [14]:
Matrix(H_fq_ext2_rref)

3×14 Matrix{fqPolyRepFieldElem}:
 1  0  0  11  1  9   7  9   11  7  6   6   1  10
 0  1  0  4   9  6   5  11  11  1  11  9   3  8
 0  0  1  4   3  12  9  10  1   4  3   12  9  10

In [15]:
Matrix(H_fq_ext1_rref)[:, (n-k+1):end]

3×10 Matrix{fqPolyRepFieldElem}:
 11  11  6   3  7   2  10  8   5   2
 5   4   10  2  10  5  2   11  11  4
 6   2   3   2  1   3  10  10  6   8

Get the corresponding generator matrices for each extended code, and then print them.

In [16]:
gm_ext1 = hcat(cvt_fq_nmod_2_int.(-Matrix(H_fq_ext1_rref)[:, (n-k+1):end])', I(k+1))
gm_ext2 = hcat(cvt_fq_nmod_2_int.(-Matrix(H_fq_ext2_rref)[:, (n-k+1):end])', I(k+2))

11×14 Matrix{Int64}:
  2   9   9  1  0  0  0  0  0  0  0  0  0  0
 12   4  10  0  1  0  0  0  0  0  0  0  0  0
  4   7   1  0  0  1  0  0  0  0  0  0  0  0
  6   8   4  0  0  0  1  0  0  0  0  0  0  0
  4   2   3  0  0  0  0  1  0  0  0  0  0  0
  2   2  12  0  0  0  0  0  1  0  0  0  0  0
  6  12   9  0  0  0  0  0  0  1  0  0  0  0
  7   2  10  0  0  0  0  0  0  0  1  0  0  0
  7   4   1  0  0  0  0  0  0  0  0  1  0  0
 12  10   4  0  0  0  0  0  0  0  0  0  1  0
  3   5   3  0  0  0  0  0  0  0  0  0  0  1

In [17]:
gm_ext1

10×13 Matrix{Int64}:
  2   8   7  1  0  0  0  0  0  0  0  0  0
  2   9  11  0  1  0  0  0  0  0  0  0  0
  7   3  10  0  0  1  0  0  0  0  0  0  0
 10  11  11  0  0  0  1  0  0  0  0  0  0
  6   3  12  0  0  0  0  1  0  0  0  0  0
 11   8  10  0  0  0  0  0  1  0  0  0  0
  3  11   3  0  0  0  0  0  0  1  0  0  0
  5   2   3  0  0  0  0  0  0  0  1  0  0
  8   2   7  0  0  0  0  0  0  0  0  1  0
 11   9   5  0  0  0  0  0  0  0  0  0  1

Check that the generator matrices of the extended Reed-Solomon codes are indeed orthogonal to their parity check matrices.

In [18]:
H_fq_ext1_rref_int = cvt_fq_nmod_2_int.(H_fq_ext1_rref)
H_fq_ext2_rref_int = cvt_fq_nmod_2_int.(H_fq_ext2_rref)
@testset begin
    @test all(iszero.(H_fq_ext1_rref_int * gm_ext1' .% q))
    @test all(iszero.(H_fq_ext2_rref_int * gm_ext2' .% q))
end;

Test Summary: | Pass  Total  Time
test set      |    2      2  2.8s


Define function to find codewords of the desired weight in the codes and test that the results agree with theoretical parameters.

In [19]:
function find_codewords_of_desired_weights(G, H, q, k, n, n_extended, wmin, wmax)
    cbs = [zeros(UInt8, 0, n+n_extended) for w in wmin:wmax]
    cu_gmsp = CuArray{UInt16}(G[:, 1:(n-k)])
    cu_H = CuArray{UInt16}(H)
    for nnmessage_nzeros = maximum([1, wmin-(n-k)]):minimum([wmax, k+n_extended])
        message_nonzeros = CuArray{UInt16}(undef, (q-1)^nnmessage_nzeros, nnmessage_nzeros)
        for i in 1:nnmessage_nzeros
            message_nonzeros[:, i] .= vcat(repeat(CUDA.fill.(1:(q-1), (q-1)^(i-1)), (q-1)^(nnmessage_nzeros-i))...)
        end
        message_array = CuArray{UInt16}(undef, (q-1)^nnmessage_nzeros, k+n_extended)
        combs = combinations(1:(k+n_extended), nnmessage_nzeros)

        for comb in combs
            message_array .= 0x0000
            message_array[:, comb] .= message_nonzeros
            
            pcs = message_array*cu_gmsp .% UInt16(q)
            cws= hcat(pcs, message_array)
            @assert all(cu_H * cws' .% UInt16(q) .== 0 )

            ws = sum(cws .!= 0, dims=2)
            ws = reshape(ws, length(ws))

            for (i, wi) in enumerate(wmin:wmax)
                cbs[i] = vcat(cbs[i], Matrix{UInt8}(cws[findall(w -> w == wi, ws), :]))
            end     
        end
    end

    @testset begin
        for (i, w) in enumerate(wmin:wmax)
            theoretical_ncws = get_num_codewords_of_weight_vn(q,q-1+n_extended,w,n-k+1)
            if theoretical_ncws > 0
                @test theoretical_ncws == size(cbs[i])[1]
                println("w$w: ", size(cbs[i])[1])
            else
                @test length(cbs[i]) == 0
            end
        end
    end

    return cbs
end

find_codewords_of_desired_weights (generic function with 1 method)

Find codebooks for the singly extended code.

In [20]:
cbs_ext1 = find_codewords_of_desired_weights(gm_ext1, H_fq_ext1_rref_int, q, k, n, 1, wmin, wmax) 

w4: 8580
Test Summary: | Pass  Total  Time
test set      |    3      3  0.0s


3-element Vector{Matrix{UInt8}}:
 0×13 Matrix{UInt8}
 0×13 Matrix{UInt8}
 [0x02 0x08 … 0x00 0x00; 0x04 0x03 … 0x00 0x00; … ; 0x00 0x00 … 0x02 0x0b; 0x00 0x00 … 0x01 0x0c]

Find codebook for the doubly extended code.

In [21]:
cbs_ext2 = find_codewords_of_desired_weights(gm_ext2, H_fq_ext2_rref_int, q, k, n, 2, wmin, wmax) 

w4: 12012
Test Summary: | Pass  Total  Time
test set      |    3      3  0.0s


3-element Vector{Matrix{UInt8}}:
 0×14 Matrix{UInt8}
 0×14 Matrix{UInt8}
 [0x02 0x09 … 0x00 0x00; 0x04 0x05 … 0x00 0x00; … ; 0x00 0x00 … 0x02 0x0b; 0x00 0x00 … 0x01 0x0c]

Save Singly extended codebooks of each requested weight.

In [22]:
for (i, w) in enumerate(wmin:wmax) #[3,4,5,6])
    if length(cbs_ext1[i]) > 0
        cbdf = DataFrame(cbs_ext1[i], "block".*string.(1:(n+1)))
        insertcols!(cbdf, 1, ("gene" => 1:nrow(cbdf)))
        CSV.write("output/RS_q"*string(q)*"_n"*string(q)*"_k"*string(k+1)*"_w"*string(w)*"cb.csv", cbdf)
        println("saved RS_q"*string(q)*"_n"*string(q)*"_k"*string(k+1)*"_w"*string(w)*"cb.csv")
    end
end;

saved RS_q13_n13_k10_w4cb.csv


Save doubly extended codebooks of each requested weight.

In [23]:
for (i, w) in enumerate(wmin:wmax) #[3,4,5,6])
    if length(cbs_ext2[i]) > 0
        cbdf = DataFrame(cbs_ext2[i], "block".*string.(1:(n+2)))
        insertcols!(cbdf, 1, ("gene" => 1:nrow(cbdf)))
        CSV.write("output/RS_q"*string(q)*"_n"*string(q+1)*"_k"*string(k+2)*"_w"*string(w)*"cb.csv", cbdf)
        println("saved RS_q"*string(q)*"_n"*string(q+1)*"_k"*string(k+2)*"_w"*string(w)*"cb.csv")
    end
end;

saved RS_q13_n14_k11_w4cb.csv


Print the parity check matrix of the singly extended codebook

In [24]:
H_fq_ext1_rref_int

3×13 Matrix{Int64}:
 1  0  0  11  11   6  3   7  2  10   8   5  2
 0  1  0   5   4  10  2  10  5   2  11  11  4
 0  0  1   6   2   3  2   1  3  10  10   6  8

and the parity check matrix of the doubly extended codebook.

In [25]:
H_fq_ext2_rref_int

3×14 Matrix{Int64}:
 1  0  0  11  1   9  7   9  11  7   6   6  1  10
 0  1  0   4  9   6  5  11  11  1  11   9  3   8
 0  0  1   4  3  12  9  10   1  4   3  12  9  10

Save the singly and doubly extended parity check matrices.

In [26]:
using DelimitedFiles
println("RS_q"*string(q)*"_n"*string(q)*"_k"*string(k+1)*"_H.csv")
open("output/RS_q"*string(q)*"_n"*string(q)*"_k"*string(k+1)*"_H.csv", "w") do io
    writedlm(io, H_fq_ext1_rref_int,",")
end

RS_q13_n13_k10_H.csv


In [27]:
using DelimitedFiles
println("RS_q"*string(q)*"_n"*string(q+1)*"_k"*string(k+2)*"_H.csv")
open("output/RS_q"*string(q)*"_n"*string(q+1)*"_k"*string(k+2)*"_H.csv", "w") do io
    writedlm(io, H_fq_ext2_rref_int,",")
end

RS_q13_n14_k11_H.csv
